In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import datetime
from time import sleep
from selenium.common.exceptions import NoSuchElementException

In [29]:
# ----------------------- Start Scrapper -----------------------

In [30]:
# Initialize Chrome
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(5)

In [31]:
# Navigate to D1D system
url = 'http://itpv2.transtron.fujitsu.com/'
driver.get(url)

In [32]:
# Fill login information
sleep(1)
input_field = driver.find_element(By.ID, 'userid')
input_field.send_keys('izmb-free')

password_field = driver.find_element(By.ID, 'password')
password_field.send_keys('izumi001')

In [33]:
# Login
sleep(1)
btn_login = driver.find_element(By.ID, 'login')
btn_login.click()

In [34]:
# Waiting the client loading
sleep(60)

In [35]:
def joinYMD(string):
    year = string.split('-')[0]
    month = string.split('-')[1]
    day = string.split('-')[2]
    return str(year) + '/' + str(month) + '/' + str(day)

In [36]:
# Get yesterday
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)

In [37]:
class color:
    SUCCESS = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'

In [38]:
# Transform the format of the scrap date
scrapDataDate = joinYMD(str(yesterday))

In [39]:
# Reset action
def resetActionKey():
    resetSpace =  driver.find_element(By.XPATH, '//*[@id="F04_mainview"]/div[2]/div[1]')
    resetSpace.click()

In [40]:
# Function to get the correct xpath of each calendar
def getCurrentCalendarXPATH(calendar_id, year, month):
    return "//table[@id='" + calendar_id + "_calendar_start_" + year + '_' + month + "']//a"

In [41]:
# Open Menu
sleep(1)
btn_menu = driver.find_element(By.CLASS_NAME, 'b_btn-menu')
btn_menu.click()

In [ ]:
# Hover Daily Report
sleep(1)
daily_report = driver.find_element(By.ID, 'F20')
hover = ActionChains(driver).move_to_element(daily_report)
hover.perform()

In [ ]:
# Navigate to Alcohol Measurement Result
sleep(1)
alcohol_measurement_result = driver.find_element(By.XPATH, '//*[@id="F25"]')
alcohol_measurement_result.click()

In [ ]:
# Cancel button to ensure the menu is turn off
# index = 10
# while 0 <= index:
#     driver.find_element(By.XPATH, '//*[@id="F04_mainview"]/div[2]/div[1]/table/tbody/tr[1]/td[2]/span[2]').click()
#     sleep(1)
#     index -= 1

In [ ]:
# Select department
sleep(1)
select_department = driver.find_element(By.ID, 'F25eigyousyo_list')
select_department.click()
sleep(1)
driver.find_element(By.XPATH, '//*[@id="F25eigyousyo_list"]/option[2]').click()

# Handle the scrapper date - should be yester day of today
# Start date
sleep(10)
select_department.click()
sleep(1)
start_date = driver.find_element(By.ID, 'F25_StartDate')
start_date.click()

index = 0

# When focus in the input of rhe calendar, the cursor is placed at the last 2 to the right
# need to move 2 steps further to move the cursor to the max right
sleep(1)
while index <= 2:
    start_date.send_keys(Keys.ARROW_RIGHT)
    index += 1

sleep(1)
while index <= 12:
    start_date.send_keys(Keys.BACKSPACE)
    index += 1

start_date.send_keys(scrapDataDate)

sleep(1)
resetActionKey()

# End date
end_date = driver.find_element(By.ID, 'F25_EndDate')
end_date.click()

index = 0

while index <= 2:
    end_date.send_keys(Keys.ARROW_RIGHT)
    index += 1

while index <= 12:
    end_date.send_keys(Keys.BACKSPACE)
    index += 1

end_date.send_keys(scrapDataDate)

sleep(1)
resetActionKey()

In [ ]:
# Options in the select department
sleep(1)
options = [item for item in select_department.find_elements(By.TAG_NAME, 'option')]
del options[0]

count = 0
existMessageBox = True
listDepartmentHaveData = []

for element in (options):
    print(f"{color.WARNING}Processing department: " + str(element.text))
    element.click()

    # Check if the selected department have data or not
    try:
        noDataNotifyModal = driver.find_element(By.ID, 'function_window_F25_messagebox_0')
    except NoSuchElementException:
        existMessageBox = False

    # If the selected departsment have data then download the data
    # otherwise, click on the OK button to turn off the modal 
    if existMessageBox == True:
        driver.find_element(By.ID, 'msg_confirm').click()
    elif existMessageBox == False:
        driver.find_element(By.ID, 'P25_csvout_button').click()
        listDepartmentHaveData.append(element.text)

    if existMessageBox == True:
        print(f"{color.FAIL}Department: " + element.text + " have no data, passed !")
    else:
        print(f"{color.SUCCESS}Department: " + element.text + " proceed successfully !")
        
    print('--------------------------------------------------------------')
    existMessageBox = True
    sleep(2)
    count += 1

print('------------------------ END SCRAPPER ------------------------')

In [ ]:
# Close the driver
index = 5
while 0 <= index:
    print("Closing the driver in: " +str(index))
    sleep(1)
    index -= 1

driver.quit()

In [ ]:
import os

directory = 'C:/Users/HaThaiViet/Downloads'

listFiles = []

for filename in os.listdir(directory):
    if filename.endswith('.zip') and filename.startswith('アルコール測定結果'): 
        listFiles.append(os.path.join(directory, filename))
        continue
    else:
        continue

In [ ]:
# Convert scrapDataDate to save as zip file name
def joinYMDZipFile(string):
    year = string.split('-')[0]
    month = string.split('-')[1]
    day = string.split('-')[2]
    return str(year) + '_' + str(month) + '_' + str(day)


In [ ]:
from zipfile import ZipFile

dateTime = joinYMDZipFile(str(yesterday))

zipObject = ZipFile('ScrapData/' + dateTime + '.zip', 'w')

zipFileName = zipObject.filename

index = 0

print(len(listFiles))

while index < len(listFiles):
    zipObject.write(listFiles[index])
    print('...added ' + str(listDepartmentHaveData[index]))
    index += 1

zipObject.close()

In [ ]:
generalZipFile = ZipFile(zipFileName, 'r')
generalZipFile.extractall('ScrapData/Extracted Files')